In [ ]:
!pip install -q -U kaggle_environments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.1/134.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
ROCK = 0
PAPER = 1
SCISSORS = 2
countForRPSCircle = 0  # Счетчик для rps_agent функции
countScissors = 0      # Счетчик для without_scissors_agent функции


# Стратегия 1
def rock_agent(observation, configuration):
    return ROCK


# Стратегия 2
def paper_agent(observation, configuration):
    return PAPER


# Стратегия 3
def scissors_agent(observation, configuration):
    return SCISSORS


# Стратегия 4
def rock_and_paper_agent(observation, configuration):
  """
    Функция ставит камень каждую 3-ю итерацию.
    В остальных случаях ставится бумага.
  """
  if observation.step % 3 == 0: return ROCK
  else: return PAPER


# Стратегия 5
def rock_and_scissors_agent(observation, configuration):
  """
    Функция ставит камень каждую 4-ю итерацию.
    В остальных случаях ставятся ножницы.
  """
  if observation.step % 4 == 0: return ROCK
  else: return SCISSORS


# Стратегия 6
def rps_agent(observation, configuration):
  """ Функция по очереди ставит ножницы, камень, а затем бумагу. """
  global countForRPSCircle
  if countForRPSCircle == 0:
    countForRPSCircle += 1
    return SCISSORS
  elif countForRPSCircle == 1:
    countForRPSCircle += 1
    return ROCK
  elif countForRPSCircle == 2:
    countForRPSCircle = 0
    return PAPER


# Стратегия 7
def copy_opponent_agent(observation, configuration):
  """ Функция копирует предыдущее поведение оппонента """
  if observation.step > 0:
      return observation.lastOpponentAction
  else:
      return random.randrange(0, configuration.signs)


# Стратегия 8
def without_scissors_agent(observation, configuration):
  """
    Если оппонент поставил ножницы 2 раза вне зависимости от порядка,
    функция ставит камень следующим ходом.
    В остальных случаях функция ставит бумагу.
  """
  global countScissors
  if countScissors == 2:
    countScissors = 0
    return ROCK
  elif observation.step > 0:
    if observation.lastOpponentAction == SCISSORS:
      countScissors += 1
      return PAPER
  else: return PAPER


# Стратегия 9
def s5p3r_agent(observation, configuration):
  """
    Функция ставит ножницы каждую 5-ю итерацию.
    Каждую третью итерацию функия ставит бумагу.
    В остальных случаях ставится камень.
  """
  if observation.step % 5 == 0: return SCISSORS
  elif observation.step % 3 == 0: return PAPER
  else: return ROCK


# Стратегия 10
def random_agent(observation, configuration):
  return random.randrange(0, 3)


# Стратегия 11
def random10_agent(observation, configuration):
  """
  Если количество итерация больше 10, функция ставит камень.
  В остальных случаях возвращаемое значение случайно.
  """
  if observation.step > 10: return ROCK
  else: return random.randrange(0, 3)


# Стратегия 12
def scissors50_agent(observation, configuration):
  """
  Если количество итерация меньше 50, функция ставит ножницы.
  В остальных случаях функция ставит бумагу.
  """
  if observation.step < 50: return SCISSORS
  else: return PAPER


# Стратегия 13
def rock90_agent(observation, configuration):
  """
  Если количество итерация больше 90, функция ставит камень.
  В остальных случаях возвращаемое значение ножницы.
  """
  if observation.step > 90: return ROCK
  else: return SCISSORS


list_of_functions = []
list_of_functions.append(rock_agent)
list_of_functions.append(paper_agent)
list_of_functions.append(scissors_agent)
list_of_functions.append(rock_and_paper_agent)
list_of_functions.append(rock_and_scissors_agent)
list_of_functions.append(rps_agent)
list_of_functions.append(copy_opponent_agent)
list_of_functions.append(without_scissors_agent)
list_of_functions.append(s5p3r_agent)
list_of_functions.append(random_agent)
list_of_functions.append(random10_agent)
list_of_functions.append(scissors50_agent)
list_of_functions.append(rock90_agent)


def compareFunc(list):
  """
  Функция для сравнения стратегий
    arg list - список со стратегиями
  """

  listResult = []  # Список с суммарными победами стратегий

  for iNum, i in enumerate(list):
    print("Идет сравнение стратегии №{}".format(iNum + 1))
    winCount = 0     # Число побед для конкретной стратегии
    for j in list:
      listTmp = evaluate("rps", [i, j], configuration={"episodeSteps": 100})
      if listTmp[0][0] > listTmp[0][1]:  # Если первая стратегия победила
        winCount += int(listTmp[0][0])
    listResult.append(winCount)
  print("\n")
  print(listResult)
  print( "Самая успешная стратегия - стратегия №{}".format( listResult.index( max( listResult ) ) + 1 ) )
  print( "Наиболее проигрышная стратегия - стратегия №{}".format( listResult.index( min( listResult ) ) + 1 ) )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
compareFunc(list_of_functions)

Идет сравнение стратегии №1
Идет сравнение стратегии №2
Идет сравнение стратегии №3
Идет сравнение стратегии №4
Идет сравнение стратегии №5
Идет сравнение стратегии №6
Идет сравнение стратегии №7
Идет сравнение стратегии №8
Идет сравнение стратегии №9
Идет сравнение стратегии №10
Идет сравнение стратегии №11
Идет сравнение стратегии №12
Идет сравнение стратегии №13


[264, 350, 216, 275, 135, 149, 0, 31, 75, 0, 238, 148, 169]
Самая успешная стратегия - стратегия №2
Наиболее проигрышная стратегия - стратегия №7
